In [ ]:
!pip install transformers accelerate datasets evaluate

In [ ]:
!pip install bitsandbytes

In [ ]:
!nvidia-smi

Thu May  9 17:13:11 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      Off | 00000000:00:03.0 Off |                    0 |
| N/A   51C    P8              12W /  72W |      1MiB / 23034MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

In [ ]:
name = 'ai-forever/ruGPT-3.5-13B'
# name = 'ai-forever/mGPT-13B'
#name = 'ai-forever/rugpt3small_based_on_gpt2'

# Loading the model and tokenizer
model = AutoModelForCausalLM.from_pretrained(
    name,
    device_map='cuda:0',
    load_in_8bit=True,
    offload_folder='./_tmp',
    max_memory={0: f'{int(torch.cuda.mem_get_info()[0] / 1024 ** 3) - 2}GB'},
    cache_dir="/content/.cache/huggingface"
)
tokenizer = AutoTokenizer.from_pretrained(name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/782 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


pytorch_model.bin.index.json:   0%|          | 0.00/42.1k [00:00<?, ?B/s]

pytorch_model-00001-of-00006.bin:   0%|          | 0.00/9.92G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/933 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.77M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.23M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/582 [00:00<?, ?B/s]

In [ ]:
import pandas as pd

df = pd.read_csv(r'./stimuli_2018_test1.csv')[['Prot', 'Sent', 'Group', 'N1', 'N2', 'Pred', 'Code', 'Sentence']]
df = df.dropna(axis=0)

In [ ]:
#df[df['Code'] == 'S_S-S']['Sentence'].to_list()

In [ ]:
import datasets
import numpy as np
import torch
from torch.nn import CrossEntropyLoss
from transformers import AutoModelForCausalLM, AutoTokenizer

import evaluate
from evaluate import logging

from tqdm.auto import tqdm

In [ ]:
def get_log_proba(text):
    all_tokens = tokenizer(f', {text},', return_tensors='pt').input_ids.to('cuda')
    prev_tokens = all_tokens[:, :-1]
    next_tokens = all_tokens[:, 1:]
    with torch.inference_mode():
        model_out = model(prev_tokens)
        tokens_log_proba = torch.gather(torch.log_softmax(model_out.logits, -1), 2, next_tokens.unsqueeze(-1))
        phrase_log_proba = tokens_log_proba.sum().item()

        #loss = model_out.loss
        #perplexity = torch.exp(loss)
        #phrase_log_proba = perplexity.item()
    return phrase_log_proba

In [ ]:
df['leftA'] = df['Sentence'].apply(lambda a: ' '.join(a.strip().split()[:4]))
df['rightA'] = df['Sentence'].apply(lambda a: ' '.join(a.strip().split()[4:]))

In [ ]:
df#['Sentence'].to_list()

,Prot,Sent,Group,N1,N2,Pred,Code,Sentence,leftA,rightA
0,1.0,1.0,1.0,S,S,S,S_S-S,Абонемент на концерт был дорогим из-за участия...,Абонемент на концерт был,дорогим из-за участия популярных артистов.
1,2.0,1.0,1.0,S,S,P,P_S-S,Абонемент на концерт были дорогими из-за участ...,Абонемент на концерт были,дорогими из-за участия популярных артистов.
2,3.0,1.0,1.0,S,P,S,S_S-P,Абонемент на концерты был дорогим из-за участи...,Абонемент на концерты был,дорогим из-за участия популярных артистов.
3,4.0,1.0,1.0,S,P,P,P_S-P,Абонемент на концерты были дорогими из-за учас...,Абонемент на концерты были,дорогими из-за участия популярных артистов.
4,5.0,1.0,1.0,P,S,S,S_P-S,Абонементы на концерт был дорогим из-за участи...,Абонементы на концерт был,дорогим из-за участия популярных артистов.
...,...,...,...,...,...,...,...,...,...,...
507,4.0,64.0,2.0,S,P,S,S_S-P,Скидка от авиакомпаний была беспрецедентной из...,Скидка от авиакомпаний была,беспрецедентной из-за недавнего крушения самол...
508,5.0,64.0,2.0,S,P,P,P_S-P,Скидка от авиакомпаний были беспрецедентными и...,Скидка от авиакомпаний были,беспрецедентными из-за недавнего крушения само...
509,6.0,64.0,2.0,P,S,S,S_P-S,Скидки от авиакомпании была беспрецедентной из...,Скидки от авиакомпании была,беспрецедентной из-за недавнего крушения самол...
510,7.0,64.0,2.0,P,S,P,P_P-S,Скидки от авиакомпании были беспрецедентными и...,Скидки от авиакомпании были,беспрецедентными из-за недавнего крушения само...


In [ ]:
df.tail(16)#['Sentence'].to_list()

,Prot,Sent,Group,N1,N2,Pred,Code,Sentence,left,right
496,1.0,63.0,2.0,P,P,S,S_P-P,Скандалы из-за статей был масштабным в глазах ...,Скандалы из-за статей,был масштабным в глазах непосредственных участ...
497,2.0,63.0,2.0,P,P,P,P_P-P,Скандалы из-за статей были масштабными в глаза...,Скандалы из-за статей,были масштабными в глазах непосредственных уча...
498,3.0,63.0,2.0,S,S,S,S_S-S,Скандал из-за статьи был масштабным в глазах н...,Скандал из-за статьи,был масштабным в глазах непосредственных участ...
499,4.0,63.0,2.0,S,S,P,P_S-S,Скандал из-за статьи были масштабными в глазах...,Скандал из-за статьи,были масштабными в глазах непосредственных уча...
500,5.0,63.0,2.0,S,P,S,S_S-P,Скандал из-за статей был масштабным в глазах н...,Скандал из-за статей,был масштабным в глазах непосредственных участ...
501,6.0,63.0,2.0,S,P,P,P_S-P,Скандал из-за статей были масштабными в глазах...,Скандал из-за статей,были масштабными в глазах непосредственных уча...
502,7.0,63.0,2.0,P,S,S,S_P-S,Скандалы из-за статьи был масштабным в глазах ...,Скандалы из-за статьи,был масштабным в глазах непосредственных участ...
503,8.0,63.0,2.0,P,S,P,P_P-S,Скандалы из-за статьи были масштабными в глаза...,Скандалы из-за статьи,были масштабными в глазах непосредственных уча...
504,1.0,64.0,2.0,P,P,P,P_P-P,Скидки от авиакомпаний были беспрецедентными и...,Скидки от авиакомпаний,были беспрецедентными из-за недавнего крушения...
505,2.0,64.0,2.0,S,S,S,S_S-S,Скидка от авиакомпании была беспрецедентной из...,Скидка от авиакомпании,была беспрецедентной из-за недавнего крушения ...


In [ ]:
df['p_w1'] = [get_log_proba(w) for w in tqdm(df.left)]
df['p_w2'] = [get_log_proba(w) for w in tqdm(df.right)]
df['p_phrase'] = [get_log_proba(w) for w in tqdm(df['Sentence'])]

  0%|          | 0/512 [00:00<?, ?it/s]

  0%|          | 0/512 [00:00<?, ?it/s]

  0%|          | 0/512 [00:00<?, ?it/s]

In [ ]:
df['compatibility_scoreA'] = df['p_phrase'] - df['p_w2A'] - df['p_w1A']

In [ ]:
df.to_csv('gpt13b_score.csv')

In [ ]:
df[['Code','p_phrase']].groupby('Code')\
            .agg(['mean','std'])

p_phrase          
            mean       std
Code                      
P_P-P -72.876953  7.648326
P_P-S -72.631836  7.774382
P_S-P -76.674805  7.527277
P_S-S -76.504395  8.525189
S_P-P -78.096680  7.803649
S_P-S -77.357910  7.823506
S_S-P -72.887207  7.310530
S_S-S -71.746582  8.957628

In [ ]:
df[['Code','compatibility_score']].groupby('Code')\
            .agg(['mean','std'])

compatibility_score          
                     mean       std
Code                               
P_P-P            8.322266  3.544970
P_P-S            9.041016  3.710921
P_S-P            5.049316  3.617763
P_S-S            4.325684  3.530931
S_P-P            3.242432  3.425112
S_P-S            4.448486  3.357410
S_S-P            8.959961  3.578687
S_S-S            9.164795  3.416710

## Simple perplexity

In [ ]:
predictions = df['Sentence'].to_list()
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
encodings = tokenizer(
    predictions,
    add_special_tokens=False,
    padding=True,
    truncation=True,
    max_length=32,
    return_tensors="pt",
    return_attention_mask=True,
    ).to('cuda')

encoded_texts = encodings["input_ids"]
attn_masks = encodings["attention_mask"]

batch_size = 12

ppls = []
loss_fct = CrossEntropyLoss(reduction="none")

for start_index in logging.tqdm(range(0, len(encoded_texts), batch_size)):
    end_index = min(start_index + batch_size, len(encoded_texts))
    encoded_batch = encoded_texts[start_index:end_index]
    attn_mask = attn_masks[start_index:end_index]

    labels = encoded_batch

    with torch.no_grad():
        out_logits = model(encoded_batch, attention_mask=attn_mask).logits

    shift_logits = out_logits[..., :-1, :].contiguous()
    shift_labels = labels[..., 1:].contiguous()
    shift_attention_mask_batch = attn_mask[..., 1:].contiguous()

    perplexity_batch = torch.exp(
        (loss_fct(shift_logits.transpose(1, 2), shift_labels) * shift_attention_mask_batch).sum(1)
        / shift_attention_mask_batch.sum(1)
    )

    ppls += perplexity_batch.tolist()

  0%|          | 0/43 [00:00<?, ?it/s]

In [ ]:
df['GPT3'] = ppls

In [ ]:
df[['Code','GPT3']].groupby('Code')\
            .agg(['mean','std'])

GPT3           
            mean        std
Code                       
P_P-P  44.629883  29.483157
P_P-S  47.082275  31.946271
P_S-P  72.231201  50.386582
P_S-S  78.085938  56.949405
S_P-P  81.658936  68.720049
S_P-S  82.544189  63.532525
S_S-P  52.054688  33.589705
S_S-S  51.966553  42.421811

In [ ]:
df.to_csv(r'./stimuli_2018_test2_alt_withpred.csv', encoding='utf8')

In [ ]:
df

## Quasi-Perplexity

In [ ]:
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased")
model = AutoModelForMaskedLM.from_pretrained("DeepPavlov/rubert-base-cased")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/714M [00:00<?, ?B/s]

In [ ]:
tokenizer("было", return_tensors="pt")

{'input_ids': tensor([[ 101, 3332,  102]]), 'token_type_ids': tensor([[0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1]])}

In [ ]:
plural = 3294 #были
singular_m = 2067 #был
singular_f = 3370 #была
singular_i = 3332 #было

buli = {'были':3294,
        'был':2067,
        'была':3370,
        'было':3332,
        }

In [ ]:
model.to('cuda')

In [ ]:
inputs = tokenizer("Абонементы на концерт [MASK] дорогими .", return_tensors="pt").to('cuda')

mask_token_index = (inputs.input_ids == tokenizer.mask_token_id)[0].nonzero(as_tuple=True)[0]

with torch.no_grad():
    logits = model(**inputs).logits

print('был --', logits[0, mask_token_index][0][singular_m])
print('были --', logits[0, mask_token_index][0][plural])

был -- tensor(6.7170, device='cuda:0')
были -- tensor(14.8506, device='cuda:0')


In [ ]:
df.head()

,Prot,Sent,Group,N1,N2,Pred,Code,Sentence,left,right
0,1.0,1.0,1.0,S,S,S,S_S-S,Абонемент на концерт был дорогим из-за участия...,Абонемент на концерт,был дорогим из-за участия популярных артистов.
1,2.0,1.0,1.0,S,S,P,P_S-S,Абонемент на концерт были дорогими из-за участ...,Абонемент на концерт,были дорогими из-за участия популярных артистов.
2,3.0,1.0,1.0,S,P,S,S_S-P,Абонемент на концерты был дорогим из-за участи...,Абонемент на концерты,был дорогим из-за участия популярных артистов.
3,4.0,1.0,1.0,S,P,P,P_S-P,Абонемент на концерты были дорогими из-за учас...,Абонемент на концерты,были дорогими из-за участия популярных артистов.
4,5.0,1.0,1.0,P,S,S,S_P-S,Абонементы на концерт был дорогим из-за участи...,Абонементы на концерт,был дорогим из-за участия популярных артистов.


In [ ]:
df[df['is_true'] == False]

In [ ]:
df['is_true'] = df['right'].apply(lambda a: a.startswith('был'))

In [ ]:
df['masked_leftA'] = df['leftA'].apply(lambda a: a+' [MASK]' )

In [ ]:
df['masked_fullA'] = df['Sentence'].apply(lambda a: ' '.join(a.strip().split()[:4])\
                                         +' [MASK] '+\
                                         ' '.join(a.strip().split()[5:]) )

In [ ]:
df['need_to_predict'] = df['rightA'].apply(lambda a: " ".join(a.strip().split()[0:]))

In [ ]:
df['need_to_predict'] = df['rightA'].apply(lambda a: a.strip().split()[0])

In [ ]:
df['need_to_predict'].value_counts()

need_to_predict
приятными      8
просторными    4
ржавыми        4
ржавым         4
привезен       4
              ..
совершены      4
повышены       4
повышен        4
возведён       4
отправлены     4
Name: count, Length: 127, dtype: int64

In [ ]:
tokenizer("было", return_tensors="pt")['input_ids'][0,1]

tensor(3332)

In [ ]:
raw_check = df.groupby(['Sent','Group','Pred']).agg({"need_to_predict":'unique'}).reset_index()
raw_check['token'] = raw_check['need_to_predict'].apply(lambda a: tokenizer(a[0], return_tensors="pt")['input_ids'][0][1:-1] )
raw_check

,Sent,Group,Pred,need_to_predict,token
0,1.0,1.0,P,[дорогими],[tensor(101534)]
1,1.0,1.0,S,[дорогим],[tensor(58020)]
2,2.0,1.0,P,[обидными],"[tensor(45371), tensor(3359)]"
3,2.0,1.0,S,[обидным],"[tensor(45371), tensor(2010)]"
4,3.0,1.0,P,[проделаны],"[tensor(35326), tensor(5242)]"
...,...,...,...,...,...
123,62.0,2.0,S,[написан],[tensor(29307)]
124,63.0,2.0,P,[масштабными],"[tensor(92662), tensor(852)]"
125,63.0,2.0,S,[масштабным],[tensor(92662)]
126,64.0,2.0,P,[беспрецедентными],"[tensor(45516), tensor(3359)]"


In [ ]:
df

,Prot,Sent,Group,N1,N2,Pred,Code,Sentence,leftA,rightA,masked_leftA,masked_fullA,need_to_predict,left_bert_is_singular,left_bert_is_singular_score,left_bert_is_plural_score
0,1.0,1.0,1.0,S,S,S,S_S-S,Абонемент на концерт был дорогим из-за участия...,Абонемент на концерт был,дорогим из-за участия популярных артистов.,Абонемент на концерт был [MASK],Абонемент на концерт был [MASK] из-за участия ...,дорогим,True,2.347879,-3.097071
1,2.0,1.0,1.0,S,S,P,P_S-S,Абонемент на концерт были дорогими из-за участ...,Абонемент на концерт были,дорогими из-за участия популярных артистов.,Абонемент на концерт были [MASK],Абонемент на концерт были [MASK] из-за участия...,дорогими,True,0.919863,-0.406250
2,3.0,1.0,1.0,S,P,S,S_S-P,Абонемент на концерты был дорогим из-за участи...,Абонемент на концерты был,дорогим из-за участия популярных артистов.,Абонемент на концерты был [MASK],Абонемент на концерты был [MASK] из-за участия...,дорогим,True,2.401935,-2.436113
3,4.0,1.0,1.0,S,P,P,P_S-P,Абонемент на концерты были дорогими из-за учас...,Абонемент на концерты были,дорогими из-за участия популярных артистов.,Абонемент на концерты были [MASK],Абонемент на концерты были [MASK] из-за участи...,дорогими,True,0.738559,-0.020243
4,5.0,1.0,1.0,P,S,S,S_P-S,Абонементы на концерт был дорогим из-за участи...,Абонементы на концерт был,дорогим из-за участия популярных артистов.,Абонементы на концерт был [MASK],Абонементы на концерт был [MASK] из-за участия...,дорогим,True,1.272758,-0.747521
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
507,4.0,64.0,2.0,S,P,S,S_S-P,Скидка от авиакомпаний была беспрецедентной из...,Скидка от авиакомпаний была,беспрецедентной из-за недавнего крушения самол...,Скидка от авиакомпаний была [MASK],Скидка от авиакомпаний была [MASK] из-за недав...,беспрецедентной,False,-0.676667,-0.676667
508,5.0,64.0,2.0,S,P,P,P_S-P,Скидка от авиакомпаний были беспрецедентными и...,Скидка от авиакомпаний были,беспрецедентными из-за недавнего крушения само...,Скидка от авиакомпаний были [MASK],Скидка от авиакомпаний были [MASK] из-за недав...,беспрецедентными,False,-0.151014,-0.151014
509,6.0,64.0,2.0,P,S,S,S_P-S,Скидки от авиакомпании была беспрецедентной из...,Скидки от авиакомпании была,беспрецедентной из-за недавнего крушения самол...,Скидки от авиакомпании была [MASK],Скидки от авиакомпании была [MASK] из-за недав...,беспрецедентной,False,-2.132132,-2.132132
510,7.0,64.0,2.0,P,S,P,P_P-S,Скидки от авиакомпании были беспрецедентными и...,Скидки от авиакомпании были,беспрецедентными из-за недавнего крушения само...,Скидки от авиакомпании были [MASK],Скидки от авиакомпании были [MASK] из-за недав...,беспрецедентными,False,-2.405480,-2.405480


In [ ]:
import numpy as np

def test_full(sentence):

    sing_ans = []

    sing = raw_check[(raw_check['Sent'] == sentence['Sent']) &
                     (raw_check['Group'] == sentence['Group']) &
                     (raw_check['Pred'] == 'S')
                     ]['token'].to_list()[0]
    print(sing)
    new_inputs = sentence['masked_leftA'][:-6]
    print(new_inputs, len(sing))
    new_inputs += ''.join(['[MASK]',]*len(sing))
    print(new_inputs)

    inputs = tokenizer(new_inputs, return_tensors="pt").to('cuda')
    mask_token_index = (inputs.input_ids == tokenizer.mask_token_id)[0].nonzero(as_tuple=True)[0]
    print(mask_token_index)

    with torch.no_grad():
        logits = model(**inputs).logits

    for enum, i in enumerate(sing):
        sing_ans.append(logits[0, mask_token_index][enum][i].detach().cpu().numpy()
                        )
    sing_ans = np.mean(sing_ans)
    print(sing_ans)

    plur_ans = []
    plur = raw_check[(raw_check['Sent'] == sentence['Sent']) &
                     (raw_check['Group'] == sentence['Group']) &
                     (raw_check['Pred'] == 'P')
                     ]['token'].to_list()[0]
    print(plur)
    new_inputs = sentence['masked_leftA'][:-6]
    print(new_inputs, len(plur))
    new_inputs += ''.join(['[MASK]',]*len(plur))
    print(new_inputs)

    inputs = tokenizer(new_inputs, return_tensors="pt").to('cuda')
    mask_token_index = (inputs.input_ids == tokenizer.mask_token_id)[0].nonzero(as_tuple=True)[0]
    print(mask_token_index)

    with torch.no_grad():
        logits = model(**inputs).logits

    for enum, i in enumerate(plur):
        plur_ans.append(logits[0, mask_token_index][enum][i].detach().cpu().numpy()
                        )
    plur_ans = np.mean(plur_ans)

    return [plur_ans, sing_ans]

    #return [logits[0, mask_token_index][0][plur].detach().cpu().numpy(),
    #        logits[0, mask_token_index][0][sing].detach().cpu().numpy(),]

dd = df.apply(lambda a: test_full(a), axis=1 )

tensor([58020])
Абонемент на концерт был  1
Абонемент на концерт был [MASK]
tensor([7], device='cuda:0')
2.347879
tensor([101534])
Абонемент на концерт был  1
Абонемент на концерт был [MASK]
tensor([7], device='cuda:0')
tensor([58020])
Абонемент на концерт были  1
Абонемент на концерт были [MASK]
tensor([7], device='cuda:0')
0.91986257
tensor([101534])
Абонемент на концерт были  1
Абонемент на концерт были [MASK]
tensor([7], device='cuda:0')
tensor([58020])
Абонемент на концерты был  1
Абонемент на концерты был [MASK]
tensor([7], device='cuda:0')
2.4019353
tensor([101534])
Абонемент на концерты был  1
Абонемент на концерты был [MASK]
tensor([7], device='cuda:0')
tensor([58020])
Абонемент на концерты были  1
Абонемент на концерты были [MASK]
tensor([7], device='cuda:0')
0.7385588
tensor([101534])
Абонемент на концерты были  1
Абонемент на концерты были [MASK]
tensor([7], device='cuda:0')
tensor([58020])
Абонементы на концерт был  1
Абонементы на концерт был [MASK]
tensor([7], device='cu

In [ ]:
def test_full(sentence):
    inputs = tokenizer(sentence['masked_left'], return_tensors="pt").to('cuda')
    mask_token_index = (inputs.input_ids == tokenizer.mask_token_id)[0].nonzero(as_tuple=True)[0]

    with torch.no_grad():
        logits = model(**inputs).logits

    return [logits[0, mask_token_index][0][buli[sentence['need_to_predict']]].detach().cpu().numpy(),
            logits[0, mask_token_index][0][plural].detach().cpu().numpy(),]

dd = df.apply(lambda a: test_full(a), axis=1 )

In [ ]:
tokenizer('беспрецедентными', return_tensors="pt")

{'input_ids': tensor([[  101, 45516,  3359,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1]])}

In [ ]:
plural = 3294 #были
singular_m = 2067 #был
singular_f = 3370 #была
singular_i = 3332 #было

def test_alt(sentence):
    inputs = tokenizer(sentence['masked_full'], return_tensors="pt").to('cuda')
    #print((inputs.input_ids == tokenizer.mask_token_id)[0].nonzero(as_tuple=True)[0][0])
    mask_token_index = (inputs.input_ids == tokenizer.mask_token_id)[0].nonzero(as_tuple=True)[0]
    #inputs = tokenizer(sentence['masked_full'], return_tensors="pt").to('cuda')

    with torch.no_grad():
        logits = model(**inputs).logits

    #print(logits[0, mask_token_index][0])

    return [logits[0, mask_token_index][0][plural].detach().cpu().numpy(),
            np.max([
            logits[0, mask_token_index][0][singular_m].detach().cpu().numpy(),
            logits[0, mask_token_index][0][singular_f].detach().cpu().numpy(),
            logits[0, mask_token_index][0][singular_i].detach().cpu().numpy()
            ]),]

dd = df.apply(lambda a: test_alt(a), axis=1 )

In [ ]:
df['left_bert_is_singular'] = [i<j for i,j in dd]

In [ ]:
df['left_bert_is_singular_score'] = [float(i[1]) for i in dd]
df['left_bert_is_plural_score'] = [float(i[0]) for i in dd]

In [ ]:
df

,Prot,Sent,Group,N1,N2,Pred,Code,Sentence,leftA,rightA,masked_leftA,masked_fullA,need_to_predict,left_bert_is_singular,left_bert_is_singular_score,left_bert_is_plural_score
0,1.0,1.0,1.0,S,S,S,S_S-S,Абонемент на концерт был дорогим из-за участия...,Абонемент на концерт был,дорогим из-за участия популярных артистов.,Абонемент на концерт был [MASK],Абонемент на концерт был [MASK] из-за участия ...,дорогим,True,2.347879,-3.097071
1,2.0,1.0,1.0,S,S,P,P_S-S,Абонемент на концерт были дорогими из-за участ...,Абонемент на концерт были,дорогими из-за участия популярных артистов.,Абонемент на концерт были [MASK],Абонемент на концерт были [MASK] из-за участия...,дорогими,True,0.919863,-0.406250
2,3.0,1.0,1.0,S,P,S,S_S-P,Абонемент на концерты был дорогим из-за участи...,Абонемент на концерты был,дорогим из-за участия популярных артистов.,Абонемент на концерты был [MASK],Абонемент на концерты был [MASK] из-за участия...,дорогим,True,2.401935,-2.436113
3,4.0,1.0,1.0,S,P,P,P_S-P,Абонемент на концерты были дорогими из-за учас...,Абонемент на концерты были,дорогими из-за участия популярных артистов.,Абонемент на концерты были [MASK],Абонемент на концерты были [MASK] из-за участи...,дорогими,True,0.738559,-0.020243
4,5.0,1.0,1.0,P,S,S,S_P-S,Абонементы на концерт был дорогим из-за участи...,Абонементы на концерт был,дорогим из-за участия популярных артистов.,Абонементы на концерт был [MASK],Абонементы на концерт был [MASK] из-за участия...,дорогим,True,1.272758,-0.747521
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
507,4.0,64.0,2.0,S,P,S,S_S-P,Скидка от авиакомпаний была беспрецедентной из...,Скидка от авиакомпаний была,беспрецедентной из-за недавнего крушения самол...,Скидка от авиакомпаний была [MASK],Скидка от авиакомпаний была [MASK] из-за недав...,беспрецедентной,True,2.954550,2.540298
508,5.0,64.0,2.0,S,P,P,P_S-P,Скидка от авиакомпаний были беспрецедентными и...,Скидка от авиакомпаний были,беспрецедентными из-за недавнего крушения само...,Скидка от авиакомпаний были [MASK],Скидка от авиакомпаний были [MASK] из-за недав...,беспрецедентными,False,1.664835,3.802301
509,6.0,64.0,2.0,P,S,S,S_P-S,Скидки от авиакомпании была беспрецедентной из...,Скидки от авиакомпании была,беспрецедентной из-за недавнего крушения самол...,Скидки от авиакомпании была [MASK],Скидки от авиакомпании была [MASK] из-за недав...,беспрецедентной,True,0.942439,0.131615
510,7.0,64.0,2.0,P,S,P,P_P-S,Скидки от авиакомпании были беспрецедентными и...,Скидки от авиакомпании были,беспрецедентными из-за недавнего крушения само...,Скидки от авиакомпании были [MASK],Скидки от авиакомпании были [MASK] из-за недав...,беспрецедентными,False,0.594528,1.754412


In [ ]:
df.to_csv('rubert_score_second.csv')

In [ ]:
df[['Code','left_bert_is_singular']].groupby('Code')\
            .agg(['mean','std'])

left_bert_is_singular          
                       mean       std
Code                                 
P_P-P              0.062500  0.243975
P_P-S              0.093750  0.293785
P_S-P              0.125000  0.333333
P_S-S              0.171875  0.380254
S_P-P              0.734375  0.445157
S_P-S              0.781250  0.416667
S_S-P              0.921875  0.270490
S_S-S              0.921875  0.270490

In [ ]:
df[['Code','full_bert_is_singular']].groupby('Code')\
            .agg(['mean','std'])

full_bert_is_singular          
                       mean       std
Code                                 
P_P-P              0.000000  0.000000
P_P-S              0.000000  0.000000
P_S-P              0.953125  0.213042
P_S-S              0.984375  0.125000
S_P-P              0.000000  0.000000
S_P-S              0.000000  0.000000
S_S-P              0.953125  0.213042
S_S-S              1.000000  0.000000

In [ ]:
df[['Code','bert_score']].groupby('Code')\
            .agg(['mean','std'])

bert_score          
            mean       std
Code                      
P_P-P   6.843250  0.996632
P_P-S   6.549697  1.327452
P_S-P   4.606411  1.638539
P_S-S   4.258953  1.669979
S_P-P   2.731027  1.586208
S_P-S   2.457126  1.564620
S_S-P   3.920699  2.973064
S_S-S   4.319447  2.908580

In [ ]:
df

,Prot,Sent,Group,N1,N2,Pred,Code,Sentence,left,right,masked_left,left_bert,masked
0,1.0,1.0,1.0,S,S,S,S_S-S,Абонемент на концерт был дорогим из-за участия...,Абонемент на концерт,был дорогим из-за участия популярных артистов.,Абонемент на концерт [MASK],True,Абонемент на концерт [MASK] дорогим из-за учас...
1,2.0,1.0,1.0,S,S,P,P_S-S,Абонемент на концерт были дорогими из-за участ...,Абонемент на концерт,были дорогими из-за участия популярных артистов.,Абонемент на концерт [MASK],True,Абонемент на концерт [MASK] дорогими из-за уча...
2,3.0,1.0,1.0,S,P,S,S_S-P,Абонемент на концерты был дорогим из-за участи...,Абонемент на концерты,был дорогим из-за участия популярных артистов.,Абонемент на концерты [MASK],True,Абонемент на концерты [MASK] дорогим из-за уча...
3,4.0,1.0,1.0,S,P,P,P_S-P,Абонемент на концерты были дорогими из-за учас...,Абонемент на концерты,были дорогими из-за участия популярных артистов.,Абонемент на концерты [MASK],True,Абонемент на концерты [MASK] дорогими из-за уч...
4,5.0,1.0,1.0,P,S,S,S_P-S,Абонементы на концерт был дорогим из-за участи...,Абонементы на концерт,был дорогим из-за участия популярных артистов.,Абонементы на концерт [MASK],False,Абонементы на концерт [MASK] дорогим из-за уча...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
507,4.0,64.0,2.0,S,P,S,S_S-P,Скидка от авиакомпаний была беспрецедентной из...,Скидка от авиакомпаний,была беспрецедентной из-за недавнего крушения ...,Скидка от авиакомпаний [MASK],False,Скидка от авиакомпаний [MASK]а беспрецедентной...
508,5.0,64.0,2.0,S,P,P,P_S-P,Скидка от авиакомпаний были беспрецедентными и...,Скидка от авиакомпаний,были беспрецедентными из-за недавнего крушения...,Скидка от авиакомпаний [MASK],False,Скидка от авиакомпаний [MASK] беспрецедентными...
509,6.0,64.0,2.0,P,S,S,S_P-S,Скидки от авиакомпании была беспрецедентной из...,Скидки от авиакомпании,была беспрецедентной из-за недавнего крушения ...,Скидки от авиакомпании [MASK],False,Скидки от авиакомпании [MASK]а беспрецедентной...
510,7.0,64.0,2.0,P,S,P,P_P-S,Скидки от авиакомпании были беспрецедентными и...,Скидки от авиакомпании,были беспрецедентными из-за недавнего крушения...,Скидки от авиакомпании [MASK],False,Скидки от авиакомпании [MASK] беспрецедентными...


In [ ]:
df['masked'] = df['Sentence'].apply(lambda a: a.replace('были','[MASK]').replace('был','[MASK]'))
df

,Prot,Sent,Group,N1,N2,Pred,Code,Sentence,left,right,masked_left,left_bert,masked
0,1.0,1.0,1.0,S,S,S,S_S-S,Абонемент на концерт был дорогим из-за участия...,Абонемент на концерт,был дорогим из-за участия популярных артистов.,Абонемент на концерт [MASK],True,Абонемент на концерт [MASK] дорогим из-за учас...
1,2.0,1.0,1.0,S,S,P,P_S-S,Абонемент на концерт были дорогими из-за участ...,Абонемент на концерт,были дорогими из-за участия популярных артистов.,Абонемент на концерт [MASK],True,Абонемент на концерт [MASK] дорогими из-за уча...
2,3.0,1.0,1.0,S,P,S,S_S-P,Абонемент на концерты был дорогим из-за участи...,Абонемент на концерты,был дорогим из-за участия популярных артистов.,Абонемент на концерты [MASK],True,Абонемент на концерты [MASK] дорогим из-за уча...
3,4.0,1.0,1.0,S,P,P,P_S-P,Абонемент на концерты были дорогими из-за учас...,Абонемент на концерты,были дорогими из-за участия популярных артистов.,Абонемент на концерты [MASK],True,Абонемент на концерты [MASK] дорогими из-за уч...
4,5.0,1.0,1.0,P,S,S,S_P-S,Абонементы на концерт был дорогим из-за участи...,Абонементы на концерт,был дорогим из-за участия популярных артистов.,Абонементы на концерт [MASK],False,Абонементы на концерт [MASK] дорогим из-за уча...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
507,4.0,64.0,2.0,S,P,S,S_S-P,Скидка от авиакомпаний была беспрецедентной из...,Скидка от авиакомпаний,была беспрецедентной из-за недавнего крушения ...,Скидка от авиакомпаний [MASK],False,Скидка от авиакомпаний [MASK]а беспрецедентной...
508,5.0,64.0,2.0,S,P,P,P_S-P,Скидка от авиакомпаний были беспрецедентными и...,Скидка от авиакомпаний,были беспрецедентными из-за недавнего крушения...,Скидка от авиакомпаний [MASK],False,Скидка от авиакомпаний [MASK] беспрецедентными...
509,6.0,64.0,2.0,P,S,S,S_P-S,Скидки от авиакомпании была беспрецедентной из...,Скидки от авиакомпании,была беспрецедентной из-за недавнего крушения ...,Скидки от авиакомпании [MASK],False,Скидки от авиакомпании [MASK]а беспрецедентной...
510,7.0,64.0,2.0,P,S,P,P_P-S,Скидки от авиакомпании были беспрецедентными и...,Скидки от авиакомпании,были беспрецедентными из-за недавнего крушения...,Скидки от авиакомпании [MASK],False,Скидки от авиакомпании [MASK] беспрецедентными...


In [ ]:
Абонемент на концерт [MASK] дорогим

был -- tensor(15.6278)
были -- tensor(7.9256)

In [ ]:
Абонемент на концерт [MASK] дорогим

был -- tensor(15.4522)
были -- tensor(8.2000)

In [ ]:
Абонементы на концерт [MASK] дорогим .

был -- tensor(9.8299)
были -- tensor(11.1843)

In [ ]:
inputs = tokenizer("Абонемент на концерт [MASK] дорогим .", return_tensors="pt")

mask_token_index = (inputs.input_ids == tokenizer.mask_token_id)[0].nonzero(as_tuple=True)[0]

with torch.no_grad():
    logits = model(**inputs).logits

topk_indices = torch.topk(logits[0, mask_token_index], k=5)

for i,j in zip(topk_indices[0], topk_indices[1]):
    print(tokenizer.decode(j),'::',i)

будет был оказался стал считается :: tensor([15.6367, 15.6278, 15.3595, 14.0973, 13.5299])


In [ ]:
def calculate_perplexity(sentence):
    inputs = tokenizer(sentence, return_tensors='pt')
    with torch.no_grad():
        outputs = model(**inputs, labels=inputs['input_ids'])
    loss = outputs.loss
    perplexity = torch.exp(loss)
    return perplexity.item()

In [ ]:
df['mdeberta-v3-base'] = df['Sentence'].apply(lambda a: calculate_perplexity(a))

In [ ]:
import pandas as pd
dff = pd.read_csv('./results_wiki4.csv')

In [ ]:
dff.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'sentence', 'root', 'root_id',
       'root_number', 'subject', 'subject_id', 'subject_number', 'distractor',
       'distractor_id', 'distractor_number', 'lemmas', 'pos', 'short_version',
       'short_version2', 'misagreed_id', 'predicate'],
      dtype='object')

In [ ]:
dff.drop(['Unnamed: 0.1', 'Unnamed: 0', 'pos', 'lemmas'],axis=1)[:300]#[['short_version','short_version2']]

,sentence,root,root_id,root_number,subject,subject_id,subject_number,distractor,distractor_id,distractor_number,short_version,short_version2,misagreed_id,predicate
0,"В случае, если между сторонами возникает спор ...",может,19,['Sing'],спор,7,['Sing'],объёме,9,['Sing'],спор об объёме может быть передан на рассмотре...,Спор об объёме может быть передан на рассмотре...,3,['может']
1,\n 815-й дальний бомбардировочный авиационный ...,награждён,30,['Sing'],полк,6,['Sing'],выполнение,9,['Sing'],\n 815-й дальний бомбардировочный авиационный...,Полк за выполнение награждён орденом « Суворов...,3,['награждён']
2,Бюджет на 2023 год и плановый период 2024 и 20...,принят,12,['Sing'],Бюджет,0,['Sing'],год,3,['Sing'],Бюджет на 2023 год был принят в ноябре 2022 го...,Бюджет на год был принят в ноябре 2022 года .,3,"['был', 'принят']"
3,Доходы бюджета на 2023 год в исходном варианте...,запланированы,9,['Plur'],Доходы,0,['Plur'],год,4,['Sing'],Доходы бюджета на 2023 год были запланированы ...,"Доходы на год были запланированы в размере 28,...",3,"['были', 'запланированы']"
4,В связи с переформированием дальней авиации в ...,введена,32,['Sing'],дивизия,22,['Sing'],дивизию,30,['Sing'],55-я авиационная дивизия дальнего действия пер...,Дивизия дивизию введена в состав 19-го бомбард...,2,['введена']
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,"Только прибытие в город подкреплений, направле...",спасло,11,['Sing'],прибытие,1,['Sing'],город,3,['Sing'],Только прибытие в город спасло Неаполь от захвата,Прибытие в город спасло Неаполь от захвата,3,['спасло']
296,"После стратегической победы при Торо, нейтрали...",возросли,24,['Plur'],нейтрализации,6,['Plur'],шансы,16,['Plur'],", нейтрализации угрозы со стороны Франции и пе...",Нейтрализации шансы возросли .,2,['возросли']
297,Список 2023 год в спорте описывает спортивные ...,описывает,6,['Sing'],Список,0,['Sing'],год,3,['Sing'],Список 2023 год описывает спортивные события 2...,Список год описывает спортивные события 2023 г...,2,['описывает']
298,"Цвет в воды в реке ярко-коричневый, так как ис...",являются,15,['Plur'],Цвет,0,['Sing'],воды,2,['Plur'],Цвет в воды являются торфяные болота . \n\n,Цвет в воды являются торфяные болота .,3,['являются']


### additional data

In [ ]:
from transformers import AutoModelForMaskedLM

def calculate_perplexity(model_name, text):
  """Calculates the perplexity score of a text using AutoModelForMaskedLM.

  Args:
    model_name: The name of the model to use.
    text: The text to score.

  Returns:
    The perplexity score of the text.
  """

  # Load the model.
  model = AutoModelForMaskedLM.from_pretrained(model_name)

  # Tokenize the text.
  tokens = model.tokenizer.tokenize(text)

  # Create a tensor of the token IDs.
  input_ids = torch.tensor(tokens).unsqueeze(0)

  # Move the input IDs to the model's device.
  input_ids = input_ids.to(model.device)

  # Calculate the output logits.
  outputs = model(input_ids, labels=input_ids)
  logits = outputs.logits

  # Calculate the perplexity.
  perplexity = torch.exp(torch.nn.functional.cross_entropy(logits, input_ids, reduction='none')).mean().item()

  return perplexity

In [ ]:
from transformers import AutoModelForMaskedLM

def calculate_textual_accuracy(model_name, text):
  """Calculates the textual accuracy score of a text using AutoModelForMaskedLM.

  Args:
    model_name: The name of the model to use.
    text: The text to score.

  Returns:
    The textual accuracy score of the text.
  """

  # Load the model.
  model = AutoModelForMaskedLM.from_pretrained(model_name)

  # Tokenize the text.
  tokens = model.tokenizer.tokenize(text)

  # Create a tensor of the token IDs.
  input_ids = torch.tensor(tokens).unsqueeze(0)

  # Move the input IDs to the model's device.
  input_ids = input_ids.to(model.device)

  # Calculate the output logits.
  outputs = model(input_ids, labels=input_ids)
  logits = outputs.logits

  # Calculate the accuracy.
  accuracy = torch.nn.functional.softmax(logits, dim=-1).argmax(dim=-1) == input_ids
  accuracy = accuracy.mean().item()

  return accuracy

In [ ]:
from transformers import AutoModelForSeq2SeqLM

def calculate_perplexity(model_name, text):
  """Calculates the perplexity score of a text using AutoModelForSeq2SeqLM.

  Args:
    model_name: The name of the model to use.
    text: The text to score.

  Returns:
    The perplexity score of the text.
  """

  # Load the model.
  model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

  # Tokenize the text.
  tokens = model.tokenizer.tokenize(text)

  # Create a tensor of the token IDs.
  input_ids = torch.tensor(tokens).unsqueeze(0)

  # Move the input IDs to the model's device.
  input_ids = input_ids.to(model.device)

  # Calculate the output logits.
  outputs = model(input_ids, labels=input_ids)
  logits = outputs.logits

  # Calculate the perplexity.
  perplexity = torch.exp(torch.nn.functional.cross_entropy(logits, input_ids, reduction='none')).mean().item()

  return perplexity